In [2]:
%%time
import pal
import json
import random
import io
import pandas as pd
from pal.prompt import math_prompts, colored_object_prompt, penguin_prompt, date_understanding_prompt, algorithmic_prompt
from tqdm.notebook import tqdm

CPU times: user 607 ms, sys: 56.2 ms, total: 663 ms
Wall time: 1.07 s


In [3]:
data = [ {"name": "Louis", "age": 7, "height (cm)": 50, "weight (kg)": 11}, {"name": "Bernard", "age": 5, "height (cm)": 80, "weight (kg)": 13}, {"name": "Vincent", "age": 9, "height (cm)": 60, "weight (kg)": 11}, {"name": "Gwen", "age": 8, "height (cm)": 70, "weight (kg)": 15} ]
df = pd.DataFrame(data)
print(df)
print(df.query("age < 8"))
print(len(df.query("age < 8")))
df = df.sort_values(by='name')
df = df.reset_index(drop=True)
print(df)
x = '{"name": "James", "age": 12, "height (cm)": 90, "weight (kg)": 12}'
print(x)
row = json.loads(x)
new_row = pd.Series(row)
df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
print(df["age"][0])

      name  age  height (cm)  weight (kg)
0    Louis    7           50           11
1  Bernard    5           80           13
2  Vincent    9           60           11
3     Gwen    8           70           15
      name  age  height (cm)  weight (kg)
0    Louis    7           50           11
1  Bernard    5           80           13
2
      name  age  height (cm)  weight (kg)
0  Bernard    5           80           13
1     Gwen    8           70           15
2    Louis    7           50           11
3  Vincent    9           60           11
{"name": "James", "age": 12, "height (cm)": 90, "weight (kg)": 12}
5


In [5]:
interface = pal.interface.ProgramInterface(
  model='gpt-4o-mini',
  stop='\n\n\n', # stop generation str for Codex API
  get_answer_expr='solution()' # python expression evaluated after generated code to obtain answer 
)

prompt = """Whenever you synthesize information and perform reasoning, it is a Thought. When you need to perform an operation on a table, it is an Action. There are 7 types of actions: 
(1) CREATE_START[data]CREATE_END: CREATE takes a data definition of a table which will be used to generate a pandas dataframe, so make sure that the input does not contain anything unnecessary for the pandas dataframe input generation. Write the data in json format.
(2) GET_VALUE_START[row_number;column]GET_VALUE_END: GET_VALUE takes the row_number, e.g., 0 is the 1st row of the table, and the name of a column, e.g., age, and returns the value of the cell, e.g., the age in the first row of the table.
(3) COUNT_START[query]COUNT_END: COUNT takes a query and counts the number of entries in the table that satisfies the query. For example, COUNT_START[age > 5]COUNT_END would count the number of entries whose value in the column "age" is larger than 5.
(4) FILTER_START[query]FILTER_END: FILTER takes a query, similarly to COUNT, and filters the entries in the table that satisfies the query. For example, FILTER_START[height < 13]FILTER_END would return the entries whose value in the column "height" is less than 13.
(5) SORT_START[column]SORT_END: SORT takes the name of a column in the table and sorts the table based on the column in lexicographical/numerical order
(6) ADD_START[row]ADD_END: ADD takes a row and adds it to the table. Write the data in json format. NOTE: This does not necessarily preserve any previous sorted order.
(7) FINISH_START[answer]FINISH_END: When you finish the question, write your answer as FINISH_START[answer]FINISH_END and terminate your response.
Terminate your response after an action. Then, an Observation is returned by the user. IMPORTANT: YOU DO NOT WRITE OBSERVATIONS. THEY WILL BE PROVIDED TO YOU. You will be given a question and you will perform Thoughts and Actions until you get the answer. When you get the answer, you will perform the Action called Finish. While you are reasoning out solutions, you will continually refer back to relevant portions of the question and employ logical reasoning to determine your next action.
"""

json_list = None
max_count = 2
counter = 0
responses = []
with open('datasets/penguins_in_a_table.json', 'r') as json_file:
    json_list = json.load(json_file)
task = json_list["task_prefix"]
json_list = json_list["examples"]
jsonSamples = random.sample(range(0, len(json_list)), max_count)
print(jsonSamples)
#jsonSamples = [819, 1000, 699, 749, 580, 366, 1275, 203, 492, 773]
#jsonSamples = [269, 436, 836, 1291, 1283, 712, 391, 474, 1164, 783]


for json_pos in tqdm(jsonSamples, desc="Going through table questions"):
    result = json_list[json_pos]
    question = result["input"]
    target = result["target_scores"]
    curResponse = {"question": question, "target": target}
    question = task + '\n' + question
    #print("Question: " + question)
    #print("Target: ", end="")
    #print(target)
    messages=[
        {"role": "system", "content": "You are a helpful assistant that accurately solves reasoning questions about tables by acting as an agent who will perform Thoughts, where you will perform logical reasoning, and Actions, which act on the tables."},
        {"role": "user", "content": prompt + '\n' + """Question: Here is a table where the first line is a header and each subsequent line is a penguin:
name, age, height (cm), weight (kg) 
Louis, 7, 50, 11
Bernard, 5, 80, 13
Vincent, 9, 60, 11
Gwen, 8, 70, 15
For example: the age of Louis is 7, the weight of Gwen is 15 kg, the height of Bernard is 80 cm. 
We now add a penguin to the table:
James, 12, 90, 12
How many penguins are less than 8 years old?"""},
        {"role": "assistant", "content": """Thought: I first need to create the table.
Action: CREATE_START[{"name": ["Louis", "Bernard", "Vincent", "Gwen"], "age": [7, 5, 9, 8], "height": [50, 80, 60, 70], "weight": [11, 13, 11, 15]}]CREATE_END
"""},
        {"role": "user", "content": "Observation: Created table"},
        {"role": "assistant", "content": """Thought: Now I need to add a new row for the new penguin
Action: ADD_START[{"name": "James", "age": 12, "height": 90, "weight": 12}]ADD_END"""},
        {"role": "user", "content": "Observation: Added row"},
        {"role": "assistant", "content": """Thought: Finally, we want to find the number of penguins whose age is less than 8.
Action: COUNT_START[age < 8]COUNT_END"""},
        {"role": "user", "content": "Observation: 2"},
        {"role": "assistant", "content": """Thought: So there are 2 penguins whose age is less than 8.
Action: FINISH_START[2]FINISH_END"""},
        {"role": "user", "content": prompt + '\n' + """Question: Here is a table where the first line is a header and each subsequent line is a penguin:
name, age, height (cm), weight (kg) 
Feynman, 43, 25, 69
Oppie, 28, 73, 42
Bob, 11, 23, 83
For example: The height of Oppie is 73 cm. What is the name of the oldest penguin?"""},
        {"role": "assistant", "content": """Thought: I first need to create the table
Action: CREATE_START[{"name": ["Feynman", "Oppie", "Bob"], "age": [43, 28, 11], "height": [25, 73, 23], "weight": [69, 42, 83]}]CREATE_END"""},
        {"role": "user", "content": "Observation: Created table"},
        {"role": "assistant", "content": """Thought: I want the oldest penguin, so first I need to sort the table by age.
Action: SORT_START[age]SORT_END"""},
        {"role": "user", "content": "Observation: Sorted table by age"},
        {"role": "assistant", "content": """Thought: Since I sorted the table, the oldest penguin must be the third row. Furthermore, I need to find their name.
Action: GET_VALUE_START[2;name]"""},
        {"role": "user", "content": "Observation: Feynman"},
        {"role": "assistant", "content": """Thought: So the oldest penguin is named Feynman.
Action: FINISH_START[Feynman]FINISH_END"""},
        {"role": "user", "content": prompt + '\n' + "Question: " + question}
    ]
    # interface.verbose = True
    finished = False
    curList = []
    #print(question)
    #print("Target: " + str(target))
    while not finished:
        gens = interface.generate_history(messages=messages, max_tokens=1024)
        output = gens[0]
        print(output)
        messages.append({"role": "assistant", "content": output})
        curAction = output[output.find("Action: ") + len("Action: "):]
        #print("Action: " + curAction)
        if "finish" in curAction.lower():
            finished = True
            expression = curAction[curAction.find("FINISH_START[") + len("FINISH_START["):curAction.find("]FINISH_END")]
            curResponse["received"] = expression
            #print(expression)
        else:
            evaled = None
            expression = None
            if "CREATE" in curAction:
                expression = curAction[curAction.find("CREATE_START[") + len("CREATE_START["):curAction.find("]CREATE_END")]
                try:
                    data = json.loads(expression)
                    curTable = pd.DataFrame(data)
                    evaled = "Created table"
                except:
                    evaled = "That was not a valid input. Try again."
                #print(curTable)
            elif "COUNT" in curAction:
                expression = curAction[curAction.find("COUNT_START[") + len("COUNT_START["):curAction.find("]COUNT_END")]
                try:
                    evaled = str(len(curTable.query(expression)))
                except:
                    evaled = "That was not a valid query. It is possible that (1) The table does not exist yet. (2) The query is invalid."
            elif "FILTER" in curAction:
                expression = curAction[curAction.find("FILTER_START[") + len("FILTER_START["):curAction.find("]FILTER_END")]
                try:
                    curTable = curTable.query(expression)
                    curTable = curTable.reset_index(drop=True)
                    evaled = "Changed the table to:\n"
                    evaled += curTable.to_string()
                except:
                    evaled = "That was not a valid query. It is possible that (1) The table does not exist yet. (2) The query is invalid."
            elif "GET_VALUE" in curAction:
                expression = curAction[curAction.find("GET_VALUE_START[") + len("GET_VALUE_START["):curAction.find("]GET_VALUE_END")]
                expression = expression.split(';')
                expression[0] = int(expression[0])
                try:
                    evaled = str(curTable[expression[1]][expression[0]])
                except:
                    evaled = "That was not a valid input. It is possible that (1) The table does not exist yet. (2) The column name is wrong."
            elif "SORT" in curAction:
                expression = curAction[curAction.find("SORT_START[") + len("SORT_START["):curAction.find("]SORT_END")]
                try:
                    curTable = curTable.sort_values(by=expression)
                    curTable = curTable.reset_index(drop=True)
                    evaled = "Sorted table by " + expression
                except:
                    evaled = "That was not a valid input. It is possible that (1) The table does not exist yet. (2) The column name is incorrect."
            elif "ADD" in curAction:
                expression = curAction[curAction.find("ADD_START[") + len("ADD_START["):curAction.find("]ADD_END")]
                try:
                    row = json.loads(expression)
                    new_row = pd.Series(row)
                    curTable = pd.concat([curTable, pd.DataFrame([new_row])], ignore_index=True)
                    evaled = "Added row."
                except:
                    evaled = "That was not a valid input. Try again."
            else:
                evaled = "That was not a valid action. Please input a valid one."
            #print("Expression: ", end = "")
            #print(expression)
            messages.append({"role": "user", "content": "Observation: " + evaled})
            #print("Evaluation: ", end="")
            #print(evaled)
            #print(messages[-1]["content"])
    curResponse["messages"] = messages
    responses.append(curResponse)
    counter += 1
numCorrect = 0

for response in responses:
    #print("Received: " + response["received"])
    #print("Target: ", end="")
    #print(response["target"])
    #print("")
    #for message in response["messages"]:
    #    print(message["content"])
    if response["received"] not in response["target"].keys():
        continue
    if response["target"][response["received"]] == 1:
        numCorrect += 1
print("Number correct = " + str(numCorrect))

[14, 65]


Going through table questions:   0%|          | 0/2 [00:00<?, ?it/s]

Thought: I first need to create the table.
Action: CREATE_START[{"name": ["Louis", "Bernard", "Vincent", "Gwen"], "age": [7, 5, 9, 8], "height": [50, 80, 60, 70], "weight": [11, 13, 11, 15]}]CREATE_END
Thought: Now I need to find the name of the penguin that is 9 years old. I can filter the table based on the age.
Action: FILTER_START[age == 9]FILTER_END
Thought: The filtered table shows that Vincent is the penguin that is 9 years old.
Action: FINISH_START[Vincent]FINISH_END
Thought: I first need to create the initial table with the given penguin data.
Action: CREATE_START[{"name": ["Louis", "Bernard", "Vincent", "Gwen"], "age": [7, 5, 9, 8], "height": [50, 80, 60, 70], "weight": [11, 13, 11, 15]}]CREATE_END
Thought: Now I need to add the new penguin, James, to the table.
Action: ADD_START[{"name": "James", "age": 12, "height": 90, "weight": 12}]ADD_END
Thought: Now I need to count the total number of animals (penguins) listed in the table.
Action: COUNT_START[]COUNT_END
Thought: I nee

In [11]:
#with open('penguin-prompting-01-results-gpt-4o-mini.json', 'w') as f:
#    json.dump(responses, f)
for response in responses:
    for message in response["messages"]:
        print(message["content"])


You are a helpful assistant that accurately solves reasoning questions about tables by acting as an agent who will perform Thoughts, where you will perform logical reasoning, and Actions, which act on the tables.
Whenever you synthesize information and perform reasoning, it is a Thought. When you need to perform an operation on a table, it is an Action. There are 7 types of actions: 
(1) CREATE_START[data]CREATE_END: CREATE takes a data definition of a table which will be used to generate a pandas dataframe, so make sure that the input does not contain anything unnecessary for the pandas dataframe input generation. Write the data in json format.
(2) GET_VALUE_START[row_number;column]GET_VALUE_END: GET_VALUE takes the row_number, e.g., 0 is the 1st row of the table, and the name of a column, e.g., age, and returns the value of the cell, e.g., the age in the first row of the table.
(3) COUNT_START[query]COUNT_END: COUNT takes a query and counts the number of entries in the table that sat

In [116]:
print(len(json_list))

149
